In [42]:
import os
from typing import Dict, List
from groq import Groq
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import time

In [ ]:
os.environ["GROQ_API_KEY"] = ""

# Models
LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
LLAMA3_8B_INSTRUCT = "llama3.1-8b-instant"
DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

# Client
client = Groq()

In [44]:
# Message Formatting
def assistant(content: str):
    return {"role": "assistant", "content": content}

def user(content: str):
    return {"role": "user", "content": content}

def system(content: str):
    return {"role": "system", "content": content}

In [45]:
# Chat Completion
def chat_completion(
    messages: List[Dict],
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    try:
        response = client.chat.completions.create(
            messages=messages,
            model=model,
            temperature=temperature,
            top_p=top_p,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return "Error generating response."

## Zero-Shot Prompting

In [49]:
def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    
    return chat_completion(
        [user(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
    )

def complete_and_print(prompt: str):
    print(f'==============\n{prompt}\n==============')
    response = completion(prompt, DEFAULT_MODEL)
    print(response, end='\n\n')

In [50]:
complete_and_print(df['human'][5])

thank you for your understanding and guidance i appreciate the reminder that i dont have to face this alone ill gather my courage and initiate that conversation soon i need to prioritize my wellbeing and find a healthier balance
It's great to hear that you're feeling more empowered to take care of yourself and prioritize your wellbeing. Remember that seeking support and guidance is a sign of strength, not weakness. 

Initiating that conversation might seem daunting, but it's often the first step towards creating positive change in your life. Take a deep breath, gather your thoughts, and remember that you're doing this for yourself and your own growth.

Finding a healthier balance is a process, and it's okay to take it one step at a time. Be kind to yourself, and don't hesitate to reach out for support when you need it. You got this, and you're not alone.



In [51]:
# Generate Few-Shot List
def random_sample(n: int, df: pd.DataFrame):
    """
    Generate a few-shot prompt list based on random samples from the dataset.
    """
    few_shot_list = []
    random_data = df.sample(n)
    human_list = random_data['human']
    gpt_list = random_data['gpt']

    # Add system message
    few_shot_list.append(system("You are a chatbot. For each message, return a response based on user input."))

    # Add user and assistant examples
    for i, j in zip(human_list, gpt_list):
        few_shot_list.append(user(i))
        few_shot_list.append(assistant(j))

    return few_shot_list

In [52]:
# Generate Response
def generate_response(few_shot_list: List[Dict], text: str):
    """
    Generate a response using the chat model.
    """
    few_shot_list.append(user(text))
    response = chat_completion(messages=few_shot_list)
    return response

In [53]:
# Evaluation Function
def evaluate_response(human_input: str, gpt_output: str, llama_response: str) -> Dict:
    """
    Evaluate the Llama response against the GPT response using semantic similarity.
    """
    model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
    embeddings = model.encode([gpt_output, llama_response])
    similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    return {
        "human_input": human_input,
        "gpt_output": gpt_output,
        "llama_response": llama_response,
        "similarity_score": similarity_score,
    }

In [54]:
def few_shot_pipeline(df: pd.DataFrame, sample_size: int, test_cases: int):
    """
    Run the few-shot prompt engineering pipeline and evaluate responses for multiple test cases.
    """
    results = []
    few_shot_list = random_sample(sample_size, df)

    test_samples = df.sample(test_cases)
    for _, row in test_samples.iterrows():
        human_input = row['human']
        gpt_output = row['gpt']

        # print(f"INPUT: {human_input}")
        # print(f"GPT Response: {gpt_output}")

        # Generate Llama response
        start_time = time.time()
        llama_response = generate_response(few_shot_list, human_input)
        elapsed_time = time.time() - start_time

        print(f"Llama Response: {llama_response}")
        print(f"Response Time: {elapsed_time:.2f} seconds")

        # Evaluate and store results
        evaluation_results = evaluate_response(human_input, gpt_output, llama_response)
        evaluation_results["response_time"] = elapsed_time
        results.append(evaluation_results)

        print(f"Evaluation Results: {evaluation_results}\n")

    return results

In [55]:
df = pd.read_csv("preprocessed_data.csv")

# Run pipeline with 10-shot prompt and 3 test case
results = few_shot_pipeline(df, sample_size=10, test_cases=3)


Llama Response: it sounds like you're feeling drained and uncertain about how to move forward when conflicts with friends affect your relationships with loved ones like your husband it can be especially challenging how have you and your husband been communicating about these conflicts are you finding any common ground or feeling like you're on the same page
Response Time: 0.99 seconds
Evaluation Results: {'human_input': 'i feel so numb its like im stuck in this emotional rut and i dont know how to get out of it my husband and i have been having so many conflicts with our friends lately and its really taking a toll on me', 'gpt_output': 'im sorry to hear that youre feeling this way charlie conflict with friends can be incredibly challenging and overwhelming can you tell me more about whats been going on with your husband and your friends', 'llama_response': "it sounds like you're feeling drained and uncertain about how to move forward when conflicts with friends affect your relationship

In [56]:
# Run pipeline with 10-shot prompt and 3 test case
results = few_shot_pipeline(df, sample_size=30, test_cases=5)

Llama Response: it sounds like you carry a heavy burden of responsibility for others happiness charlie remember that you cant control how others feel or react you can only control your own actions and choices it might be helpful to explore where this sense of responsibility comes from and how it affects your relationships
Response Time: 10.42 seconds
Evaluation Results: {'human_input': 'well alex i constantly worry about letting people down or not being there for them in the way they need its as if their happiness and fulfillment are solely dependent on my actions and choices this pressure makes me question my worth and ability to maintain healthy relationships', 'gpt_output': 'it sounds like you place a lot of importance on your role in maintaining the happiness of others charlie have you ever considered whether this responsibility youre carrying is realistic or whether there might be a more balanced way to approach personal relationships', 'llama_response': 'it sounds like you carry 

## Few Shot With Chain of Thought(CoT)

In [57]:
# Generate Few-Shot List with CoT Example
def random_sample_cot(df: pd.DataFrame, n: int) -> List[Dict]:
    """
    Generate a few-shot prompt list using CoT examples.
    """
    few_shot_list = []
    random_data = df.sample(n)
    human_list = random_data['human']
    gpt_list = random_data['gpt']  # Ensure this column contains CoT responses

    # Add system message
    few_shot_list.append(system("You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries."))

    # Add user and assistant examples
    for human, gpt in zip(human_list, gpt_list):
        few_shot_list.append(user(human))
        few_shot_list.append(assistant(gpt))

    return few_shot_list

In [58]:
def few_shot_with_cot_pipeline(df: pd.DataFrame, sample_size: int, test_cases: int):
    """
    Run the few-shot prompt engineering pipeline and evaluate responses for multiple test cases.
    """
    results = []
    few_shot_list = random_sample_cot(df,sample_size)
    print(few_shot_list)
    test_samples = df.sample(test_cases)
    for _, row in test_samples.iterrows():
        human_input = row['human']
        gpt_output = row['gpt']

        # print(f"INPUT: {human_input}")
        # print(f"GPT Response: {gpt_output}")

        # Generate Llama response
        start_time = time.time()
        llama_response = generate_response(few_shot_list, human_input)
        elapsed_time = time.time() - start_time

        # print(f"Llama Response: {llama_response}")
        # print(f"Response Time: {elapsed_time:.2f} seconds")

        # Evaluate and store results
        evaluation_results = evaluate_response(human_input, gpt_output, llama_response)
        evaluation_results["response_time"] = elapsed_time
        results.append(evaluation_results)

        # print(f"Evaluation Results: {evaluation_results}\n")

    return results

In [ ]:
results = few_shot_with_cot_pipeline(df, 5, 2)
results

[{'role': 'system', 'content': 'You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries.'}, {'role': 'user', 'content': 'i think these walls prevent me from fully expressing myself and being vulnerable they create a barrier between me and potential partners making it difficult for them to truly get to know me its like im holding myself back from experiencing love again'}, {'role': 'assistant', 'content': 'it sounds like you understand how these walls might be hindering your ability to form meaningful connections breaking down those barriers takes time and selfwork what steps do you think you can take to gradually let go of this defense mechanism'}, {'role': 'user', 'content': 'well i think practicing selfcompassion and challenging negative selftalk could be beneficial sometimes im my own worst critic and i need to learn to be gentler with myself'}, {'role': 'assistant', 'content': 'absolutely charlie selfcompassion is powerful it helps us shift 

[{'human_input': 'yes alex the biggest challenge im facing right now is letting go of the past and fully embracing the future sometimes thoughts of what could have been or what went wrong creep in and its disheartening however im actively working on redirecting those thoughts and focusing on the present moment',
  'gpt_output': 'letting go of the past is undoubtedly a challenging process charlie its great to hear that youre actively working on redirecting your thoughts and staying present remember healing takes time and its okay to have moments where you stumble the key is to show yourself compassion and continue moving forward',
  'llama_response': "that takes a lot of courage charlie acknowledging the past and its impact on your present is a huge step in the healing process it sounds like you're practicing mindfulness by redirecting those thoughts and focusing on the present moment that's a great start now lets think about what steps you can take to nurture a sense of closure and acc

In [ ]:
results = few_shot_with_cot_pipeline(df, 20, 10)
results

[{'role': 'system', 'content': 'You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries.'}, {'role': 'user', 'content': 'i have started attending support groups and therapy sessions to get professional help im also trying to be more open and honest with my family about my struggles'}, {'role': 'assistant', 'content': 'its commendable that youve taken these steps and are actively seeking support how have your family members reacted to your efforts'}, {'role': 'user', 'content': 'thank you alex your support means a lot to me in our next session id like to explore strategies for maintaining motivation and overcoming obstacles during moments of uncertainty or setback i believe having strategies in place will help me stay on track and continue growing even when faced with challenges'}, {'role': 'assistant', 'content': 'thats a great area to focus on charlie we can explore various techniques and tools to cultivate resilience and maintain motivation du

[{'human_input': 'thank you alex i appreciate your guidance and support i think weve covered a lot of ground today and i feel more confident in my ability to handle conflict in my friendships ill definitely keep working on implementing what we discussed right now im feeling happy and hopeful for stronger more resilient friendships',
  'gpt_output': 'im glad to hear that charlie remember that growth takes time and its normal to have ups and downs celebrate your progress and dont be too hard on yourself when challenges arise if theres anything else youd like to discuss in the future or any other topics that come up know that im here to listen and support you along the way',
  'llama_response': 'that warms my heart charlie its been an absolute pleasure working with you and watching you grow its clear that you value your friendships and are committed to nurturing them remember relationships are a two-way street and its okay to make mistakes along the way the important thing is that youre w

## Trying CoT with updating the human and gpt response 

In [59]:
# Function to add Chain of Thought reasoning to GPT responses
def apply_cot_prompt_engineering(human_input, gpt_response):
    """
    Transform GPT responses into a Chain of Thought (CoT) format.
    """
    return (
        f"Step 1: Understand the user's input.\n"
        f"User Input: {human_input}\n\n"
        f"Step 2: Address the user's main concern.\n"
        f"{gpt_response.split('.')[0]}.\n\n"
        f"Step 3: Provide a clear solution or suggestion.\n"
        f"{gpt_response.split('.')[1] if len(gpt_response.split('.')) > 1 else 'Consider reflecting on this further.'}"
    )

# Apply CoT transformation to each row
df['gpt_cot'] = df.apply(
    lambda row: apply_cot_prompt_engineering(row['human'], row['gpt']),
    axis=1
)


In [60]:
df.to_csv('final_preprocessed.csv')

In [62]:
df.iloc[0]

Unnamed: 0                                                    0
id                                           identity_0_chunk_1
human         ive been feeling so sad and overwhelmed lately...
gpt           hey there im here to listen and support you it...
gpt_cot       Step 1: Understand the user's input.\nUser Inp...
Name: 0, dtype: object

In [ ]:
# Generate Few-Shot List with CoT Example
def random_sample_cot(df: pd.DataFrame, n: int) -> List[Dict]:
    """
    Generate a few-shot prompt list using CoT examples.
    """
    few_shot_list = []
    random_data = df.sample(n)
    human_list = random_data['human']
    gpt_list = random_data['gpt_cot']  # Ensure this column contains CoT responses

    # Add system message
    few_shot_list.append(system("You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries."))

    # Add user and assistant examples
    for human, gpt in zip(human_list, gpt_list):
        few_shot_list.append(user(human))
        few_shot_list.append(assistant(gpt))

    return few_shot_list

In [ ]:
def few_shot_with_cot_pipeline(df: pd.DataFrame, sample_size: int, test_cases: int):
    """
    Run the few-shot prompt engineering pipeline and evaluate responses for multiple test cases.
    """
    results = []
    few_shot_list = random_sample_cot(df,sample_size)
    print(few_shot_list)
    test_samples = df.sample(test_cases)
    for _, row in test_samples.iterrows():
        human_input = row['human']
        gpt_output = row['gpt_cot']

        # print(f"INPUT: {human_input}")
        # print(f"GPT Response: {gpt_output}")

        # Generate Llama response
        start_time = time.time()
        llama_response = generate_response(few_shot_list, human_input)
        elapsed_time = time.time() - start_time

        # print(f"Llama Response: {llama_response}")
        # print(f"Response Time: {elapsed_time:.2f} seconds")

        # Evaluate and store results
        evaluation_results = evaluate_response(human_input, gpt_output, llama_response)
        evaluation_results["response_time"] = elapsed_time
        results.append(evaluation_results)

        # print(f"Evaluation Results: {evaluation_results}\n")

    return results

In [ ]:
results = few_shot_with_cot_pipeline(df, 5, 2)
results

[{'role': 'system', 'content': 'You are a chatbot that uses step-by-step reasoning (Chain of Thought) to respond to user queries.'}, {'role': 'user', 'content': 'i havent considered that but it sounds like a helpful idea i think having a support system and someone who understands what im going through would make a significant difference thank you for suggesting that alex'}, {'role': 'assistant', 'content': "Step 1: Understand the user's input.\nUser Input: i havent considered that but it sounds like a helpful idea i think having a support system and someone who understands what im going through would make a significant difference thank you for suggesting that alex\n\nStep 2: Address the user's main concern.\nyoure welcome charlie building a support system is crucial in navigating through challenging times surrounding yourself with understanding and empathetic individuals can provide a sense of validation and comfort remember healing takes time but you dont have to face it alone.\n\nSte

[{'human_input': 'maybe i could remind myself of the unique skills and strengths i bring to the table instead of focusing on what i lack i can concentrate on the value i provide to the team it would require me to nurture selfacceptance and practice selfcompassion',
  'gpt_output': "Step 1: Understand the user's input.\nUser Input: maybe i could remind myself of the unique skills and strengths i bring to the table instead of focusing on what i lack i can concentrate on the value i provide to the team it would require me to nurture selfacceptance and practice selfcompassion\n\nStep 2: Address the user's main concern.\nthats an excellent suggestion charlie cultivating selfacceptance and practicing selfcompassion are powerful tools to counteract feelings of jealousy how would you envision implementing these practices in your daily work life.\n\nStep 3: Provide a clear solution or suggestion.\nConsider reflecting on this further.",
  'llama_response': 'Step 1: Understand the user\'s input.\

In [ ]:
for i in results:
    print(i['llama_response'].split("\n")[-1])

Consider creating a "strengths journal" where you write down three things you do well and three things you're proud of accomplishing each day this can help you develop a more positive and compassionate mindset towards yourself and remind you of your unique value and strengths.
Consider taking small steps towards opening up to her start with sharing something personal or a vulnerable experience and see how she responds this can help you gauge her willingness to reciprocate and deepen your connection.


In [ ]:
results[0]['llama_response'].split("\n")[-1]

'Consider creating a "strengths journal" where you write down three things you do well and three things you\'re proud of accomplishing each day this can help you develop a more positive and compassionate mindset towards yourself and remind you of your unique value and strengths.'


<h1> Directional Stimulus Prompting (DSP) </h1> is a prompt engineering approach used to guide language models toward specific outputs by including explicit constraints or desired styles in the prompt. This can be particularly useful for tasks like generating structured responses, enforcing reasoning styles, or encouraging creativity.

In [ ]:
# Function to create DSP prompts
def gpt_dsp(human_input, hint="empathetic and structured"):
    """
    Generate a Directional Stimulus Prompt (DSP) for the dataset.
    """
    # Add a directional stimulus to the prompt
    dsp_prompt = (
        f"Hint: Respond to the following input in an {hint} manner, "
        f"providing a structured, step-by-step explanation.\n"
        f"User Input: {human_input}\n"
    )
      
    return dsp_prompt



In [ ]:
def dsp_pipeline(df: pd.DataFrame, test_cases: int):
    """
    Run the dsp prompt engineering pipeline and evaluate responses for multiple test cases.
    """
    results,few_shot_list = [], []
    eval_dict = {}
    test_samples = df.sample(test_cases)
    for _, row in test_samples.iterrows():
        human_input = row['human']

        # Generate Llama response
        start_time = time.time()
        llama_response = generate_response(few_shot_list, gpt_dsp(human_input))
        print("\n\n")
        print("human input", human_input)
        print("llama response", llama_response)
        
        elapsed_time = time.time() - start_time

        # Evaluate and store results
        # evaluation_results = evaluate_response(human_input, llama_response)
        eval_dict['llama_response'] = llama_response
        eval_dict["response_time"] = elapsed_time

        results.append(eval_dict)
        
        

        # print(f"Evaluation Results: {evaluation_results}\n")

    return results

In [ ]:
dsp_pipeline(df, 2)




human input i think that would be really helpful maybe i can start by setting small achievable goals and gradually work my way up that way i wont feel overwhelmed
llama response I completely understand your concerns and willingness to take the first step.  Setting small, achievable goals is an excellent approach to build momentum and confidence. Here's a step-by-step guide to help you get started:

**Step 1: Identify Your Goals**
Take some time to reflect on what you want to achieve. Be specific and make sure your goals are aligned with your values and priorities. Write down your goals and make them concrete.

**Step 2: Break Down Big Goals into Smaller Ones**
Divide your larger goals into smaller, manageable tasks. This will help you create a sense of accomplishment and progress. For example, if your goal is to learn a new language, break it down into smaller tasks like learning a new vocabulary word each day or practicing for 10 minutes each morning.

**Step 3: Make Your Goals SMA

[{'llama_response': "I can sense the positivity and resilience in your words, and it's truly inspiring. It's amazing how you're finding joy and strength in each small victory, and I'm here to support and guide you through this journey.\n\nIt sounds like you're already on the right path, and I'd like to help you build upon that momentum. Here's a structured approach to help you continue celebrating your small victories and working towards your larger goals:\n\n1. **Reflect on your successes**: Take a moment to reflect on the small victories you've achieved so far. What did you accomplish, and how did it make you feel? Writing down your successes can help you identify patterns and areas where you're making progress.\n\n2. **Identify the next steps**: Think about what you'd like to achieve next. Break down larger goals into smaller, manageable tasks. What's the next step you can take towards your goal? Make sure it's specific, measurable, and achievable.\n\n3. **Create a celebration plan*